In [1]:
import pandas as pd
import re
import nltk
import re
from stopwordsiso import stopwords as stopwords_iso
from nltk.corpus import stopwords as stopwords_nltk
# from langdetect import detect
import stanza
from tqdm import tqdm
from deep_translator import GoogleTranslator
from langdetect import detect, LangDetectException
from lingua import Language, LanguageDetectorBuilder
# from transformers import pipeline

c:\Users\rainn\miniconda3\envs\mainenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\rainn\miniconda3\envs\mainenv\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
# PROCESSORS
languages = [Language.ENGLISH, Language.TAGALOG]
detector = LanguageDetectorBuilder.from_languages(*languages).build()
stanza.download('en')
# TAGALOG TOKENIZER FOR SENTENCES THAT ARE DOMINANT IN TAGALOG WORDS (used to segment sentences into a bag of words)
nlp = stanza.Pipeline('en', processors="tokenize,lemma",use_gpu=True, treebank='ESLSpok')
# STOP WORDS for both languages
nltk.download('stopwords')
stopwords_all = stopwords_nltk.words('english') + list(stopwords_iso('tl')) + ["im", "i'm", "i am"]

# translator
# translator = GoogleTranslator(source='auto', target='en')

# pipeline for bert transformer
# bert_pipe_tiai = pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")

2025-02-22 18:54:22 INFO: Downloaded file to C:\Users\rainn\stanza_resources\resources.json
2025-02-22 18:54:22 INFO: Downloading default packages for language: en (English) ...
2025-02-22 18:54:23 INFO: File exists: C:\Users\rainn\stanza_resources\en\default.zip
2025-02-22 18:54:26 INFO: Finished downloading models and saved to C:\Users\rainn\stanza_resources
2025-02-22 18:54:26 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-02-22 18:54:26 INFO: Downloaded file to C:\Users\rainn\stanza_resources\resources.json
2025-02-22 18:54:26 WARNING: Language en package default expects mwt, which has been added
2025-02-22 18:54:26 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| lemma     | combined_nochar

In [3]:
tagalog_contractions = {
    'sya': 'siya',
    'bat': 'bakit',
    'ikay': 'ikaw ay',
    'sakin': 'sa akin',
    'sayo': 'sa iyo',
    'samin': 'sa amin',
    'withraw':'withdraw',
    'satin': 'sa atin',
    'niyo':'ninyo',
    'nyo': 'ninyo',
    'nang': 'na ang',
    'tas': 'tapos',
    'meron': 'mayroon',
    'penge': 'pahingi',
    'di' : 'hindi',
    'pls': 'please',
    'sna': 'sana',
    'gomana': 'gumana',
    'dili': 'hindi',
    'hoi': 'hey',
    'dko': 'hindi ko',
    'nmn':'',
    'naman':'',
    'nman':'',
    'kaka':'',
    'bkt': 'bakit',
    'prin': 'parin',
    'nkaka':'',
    'pla':'pala',
    '2fa': 'authentication',
    'transac': 'transaction',
    'app': 'application',
    'nah':'no',
    'pesos':'peso',
    'php':'peso',
    'pera':'money',
    'apps':'application',
    'bwesit':'bwisit',
    'bwiset':'bwisit',
    'tngina':'putangina',
    'tangina':'putangina',
    'tnginaka':'putangina',
    'kingina':'putangina',
    'mbgal':'bagal',
    'mabagal':'bagal',
    'gcahs':'gcash',
    'gagu':'gago',
    'bubu':'bobo',
    'ogag':'gago',
    'obob':'bobo',
    'tgal':'matagal',
    'tagal':'matagal',
    'mtagal':'matagal',
    'accnt':'account',
    'recieve':'receive',
    'recieved':'receive',
    'ive':'have',
    'ayosin':'ayusin',
    'q': 'ko',
    'xa':'siya',
    'ug':'iyon',
    'yung':'iyon',
    'yong':'iyon',
    'imong':'iyon',
    'hindot':'putangina',
    'lage':'palagi',
    'laging':'palagi',
    'lagi':'palagi',
    'palaging':'palagi',
    'bkit':'bakit',
    'lyk':'like',
    'dis':'this',
    'nko':'nako',
    'de':'hindi',
    'dl':'download',
    'cp':'phone',
    'cellphone':'phone',
    'ai':'artificial intelligence',
    'nd':'',
    'ndi':'hindi',
    'hndi':'hindi',
    'andami':'mada',
    'pinag':'',
    'kapag':'',
    'naka':'',
    'nakaka':'',
    'nkka':'',
    'dabest':'best', 
    'pakyu':'putangina',
    'ampanget':'pangit',
    'panget':'pangit',
    'hina':'mahina',
    
}
negation_words = [
    "not", "no", "never", "none", "nothing", "nowhere", "neither", "nor", "without",
    "don't", "do not", "doesn't", "does not", "didn't", "did not",
    "can't", "cannot", "couldn't", "could not", "won't", "will not",
    "wouldn't", "would not", "shouldn't", "should not",
    "isn't", "is not", "aren't", "are not", "wasn't", "was not", "weren't", "were not",
    "hasn't", "has not", "haven't", "have not", "hadn't", "had not",

    "hindi", "hnd", "ndi", "wala", "walang",
    "ayaw", "wag", "huwag", "hnd pa", "ndi pa", "di pa", "wala pa",
    "huwag na", "hnd pa", "hindi pa", "ayoko", "di ako", "wala nang",
    "di mo", "d mo", "di siya", "d siya", "di sila",
    "hindi rin", "hnd rin", "hindi naman", "wala ito", "wala akong"
]


In [4]:
# PLAYGROUND
    # stopwords_nltk.words('english')
# print(bert_pipe_tiai("Putanginaaaa mong gcash ka andami konang Pera nagloko pa madaya yang putanginang yan Wag nyo to i, install putangggina"))
# print(bert_pipe_dost("Putanginaaaa mong gcash ka andami konang Pera nagloko pa madaya yang putanginang yan Wag nyo to i, install putangggina"))
# print(bert_pipe_dost("this is not good gchash"))

In [5]:
# LOAD DF ONLY GET IMPORTANT COLUMNS
load_list = ['2020-','2021-','2022']
def load_df(load):
    # removes words that are less than 3 letters, no letters, stopwords, contractions, unnecessary columns, and null values
    data = pd.read_csv('GCASH_REVIEWS.csv')
    data = data[data['review_text'].str.contains(r'[a-zA-Z]', na=False)]
    data = data[data['review_text'].str.len() > 3]
    data = data[~data['review_text'].isin(tagalog_contractions)]
    data = data[~data['review_text'].isin(stopwords_all)]
    data = data.drop(columns=['author_app_version', 'author_id', 'author_name'])
    data = data.dropna()
    original = data.copy()
    # FILTER BY YEAR just change 20XX-
    data = data[data['review_datetime_utc'].str.startswith(load)]
    return data, original

# df, df_original = load_df()

In [6]:
# 'text' is one row of string of words for every row in df['review_text']
def handleNegation(text: str) -> str:
    for neg in negation_words:
        text = text.replace(" "+neg+" ", " negativelabel ")
    return text

label_obj = {"Very Negative" : 'verynegativelabel', "Negative": 'negativelabel', "Neutral": 'neutrallabel', "Positive": 'positivelabel', "Very Positive": 'verypositivelabel'}
def clean_text(text: str):
    
    # handles negation identification and adds negative label as placeholder to not be removed in stopwords
    text = text.lower()
    text_negation_handled = handleNegation(text)
    
    # filters text, remove special characters but keeps hyphened words
    # text = re.sub(r"\b(nkka|nakaka|nka|naka|nagka|nagkaka|magka|magkaka|mgka|mgkaka|)\w*\b", "", text_negation_handled)
    text = re.sub(r"[^a-zA-Z0-9]", " ", text_negation_handled)
    text = text.replace("g cash", "gcash")
    
    # turn tagalogs into their non contraction form if available (e.g satin -> sa atin)
    words = text.split()
    non_contracted_words = [(tagalog_contractions[w] if w in tagalog_contractions else w) for w in words]

    # exclude stopwords for english and tagalog (e.g ang, may, ako, myself, is, the)
    no_stop_words = [ncw for ncw in non_contracted_words if ncw not in stopwords_all]

    # do lemmatizing to identify words of context
    filtered_words = ' '.join(no_stop_words)
    filtered_words = filtered_words.lower()

    doc = nlp(filtered_words)
    
    principal_words = []
    for sentence in doc.sentences:
        for word in sentence.words:
            w = word.lemma if word.lemma else word.text
            # print(w, detect(w))
            if len(w) < 2 or w.isdigit(): 
                continue
            # if detect(w) in ['en', 'tl']: 
            principal_words.append(w)
    
    # printouts
    # print("negation handled: ", text_negation_handled)
    # print("regex applied: ", text)
    # print("noncontractedwords ", non_contracted_words)
    # print("no stop words", no_stop_words)
    # print("filtered_words ", filtered_words)
    # print(principal_words)
    # then return an array of words
    return principal_words if len(principal_words) > 2 else None
            
# clean_text("Ok 🎉 sya gamitin ung SSS .but sometimes may difficulties,...but worthit ..na intact parin yung laman ng gcash? 😂")
    
    


In [ ]:
for load in load_list:
    df, _ = load_df(load)
    df['review_text_formatted'] = [clean_text(text) for text in tqdm(df['review_text'])]
    df.to_csv('gcash-{load}formatted.csv')

In [ ]:
# df_sample = df.sample(100)
# df_sample['review_text_formatted'] = [clean_text(text) for text in tqdm(df_sample['review_text'])]